In [1]:
#==街景字体识别==
#思路：将不定长字符转换为定长字符的识别问题，并使用CNN完成训练和验证


# task1 任务理解

In [ ]:
# Day1 赛题理解：

# 任务目标：题目是街景字符编码识别。

# ………………………………………………………………
# 数据文件下载查看，有六个：

# 提交的模板，三个划分好的数据集，train,val,test，并在两个.json文件中给出了train和val中目标的定位。可学习定位。

# 000000.png": {
# "height": [219, 219], "label": [1, 9], "left": [246, 323], "top": [77, 81], "width": [81, 96]
# }

# 长、宽、到左到上坐标、图中数字

# 。

# 单张图查看：

# 图片中以矩形框画出了数字区域。
# ………………………………………………………………

# 思路：

# 已知数字位置，做10分类问题；不知道，做11分类问题（0~9&null ）；也可以做识别问题，先定位再分类。
# ……………………………………………………………

# 技巧：

# 观察题目中的图像，有不同角度的偏转，那么数据集应该需要进行一定的扩增了，至少左右45度的旋转是少不了了，再结合具体问题，数字作为特定的图像符号，不能进行翻转等方法进行扩增。

# ………………………………………………………………

# 步骤：

# 参考minist 数据集手写字体识别。先做一个baseline。

# ………………………………………………………………

# 其他思路参考：

# 图片中出现的数字的个数并不确定，考虑不定长的数字识别。或做字符串检测，然后再在字符串里做分割识别？或者就是单纯的进行数字识别再分类？ 膨胀腐蚀。
# Whale提供了思路，街景字符编码识别 GitHub链接，用定长的字符段去解决的。

# ==task2 数据读取与扩增==

In [218]:
from PIL import Image
import cv2
#读图片库Pillow,OpenCV等



In [ ]:
#测试读单张图
#OpenCV imread imwrite
#pillow Image.open() im.save()
img = cv2.imread("data//mchar_train//000000.png")
img#读出来的是图片的象素数据
'''
array([[[ 98, 112, 108],
        [ 97, 112, 108],
        [ 98, 114, 107],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],
'''

In [220]:
cv2.imwrite('000.jpg',img)#写出cv2.imwrite

True

In [221]:
edges = cv2.Canny(img,30,70)#cv2.Canny边缘检测
cv2.imwrite('first.jpg',edges)

True

In [222]:
imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)#转换为灰度图2RGB，由BRG转为RGB
cv2.imwrite('000gray.jpg',imgGray)

True

In [223]:
#数据扩增:对图片做变换，翻转缩放颜色亮度等变化，增加数据，
#常用数据扩增库：
#torchvision 
# https://github.com/pytorch/vision
# pytorch官方提供的数据扩增库，提供了基本的数据数据扩增方法，可以无缝与torch进行集成；但数据扩增方法种类较少，且速度中等；
#imgaug 
#https://github.com/aleju/imgaug
#imgaug是常用的第三方数据扩增库，提供了多样的数据扩增方法，且组合起来非常方便，速度较快；
#albumentations
#https://albumentations.readthedocs.io
#是常用的第三方数据扩增库，提供了多样的数据扩增方法，对图像分类、语义分割、物体检测和关键点检测都支持，速度较快。
'''
transforms.CenterCrop 对图片中心进行裁剪
transforms.ColorJitter 对图像颜色的对比度、饱和度和零度进行变换
transforms.FiveCrop 对图像四个角和中心进行裁剪得到五分图像
transforms.Grayscale 对图像进行灰度变换
transforms.Pad 使用固定值进行像素填充
transforms.RandomAffine 随机仿射变换
transforms.RandomCrop 随机区域裁剪
transforms.RandomHorizontalFlip 随机水平翻转
transforms.RandomRotation 随机旋转
transforms.RandomVerticalFlip 随机垂直翻转
'''

'\ntransforms.CenterCrop 对图片中心进行裁剪\ntransforms.ColorJitter 对图像颜色的对比度、饱和度和零度进行变换\ntransforms.FiveCrop 对图像四个角和中心进行裁剪得到五分图像\ntransforms.Grayscale 对图像进行灰度变换\ntransforms.Pad 使用固定值进行像素填充\ntransforms.RandomAffine 随机仿射变换\ntransforms.RandomCrop 随机区域裁剪\ntransforms.RandomHorizontalFlip 随机水平翻转\ntransforms.RandomRotation 随机旋转\ntransforms.RandomVerticalFlip 随机垂直翻转\n'

In [224]:
#Pytorch读取数据
#Dataset封装数据集，配置DataLoder【设置 迭代器 】封装批量读取迭代方法


In [225]:
# Pytorch安装,参考https://www.cnblogs.com/zhouzhiyao/p/11784055.html
# 1、创建虚拟环境
# conda create -n pytorch python=3.6
# //安装后查看虚拟环境
# conda info --envs
# 3、激活
# conda activate pytorch
# 4、安装pytorch
# 到官网，自行选择电脑配置生成安装路径，不用装CUDA，选好自动配好代码
# //包windows建议conda，linux建议选pip
# //pytorch Build 选stable
# //cuda 显卡 有GPU的话 python3.6选9.2，没有选None
# //生成结果：conda install pytorch torchvision cudatoolkit=9.2 -c pytorch -c defaults -c numba/label/dev
# 粘到conda下pytorch虚拟环境安装即可


In [226]:
import os,sys,glob,shutil,json
os.environ["CUDA_VISIBLE_DEICES"] = '0'
import cv2
from PIL import Image
import numpy as np
import torch
from torch.utils.data.dataset import Dataset
import torchvision.transforms as transforms
import torchvision.models as models
import pickle

torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True


In [227]:
#Dataset封装数据集
class SVHNDataset(Dataset):
    def __init__(self,img_path,img_label,transform = False):#__init__后面__别忘了 transform = None
        self.img_path = img_path
        self.img_label = img_label
        self.transform = transform
#         if transform is not None:
#             self.transform = transform
#         else:
#             self.transform = None
            
    def __getitem__(self,index):
        img = Image.open(self.img_path[index]).convert('RGB')#pillow包 按索引打开图片
        if self.transform:
            img = self.transform(img)#数据转换
        #//分10类 SVHN中类别10是0
        #设置最长字符长度为5-观察图一般5个字符，不够pad
        #按索引读数据结果标签
        lbl = np.array(self.img_label[index],dtype = np.int)
        #5-字符长 乘除的数字个数*[10] 即长度小于5，则少几位就加几个类别10来占位，10类的值为0
        lbl = list(lbl)+(5-len(lbl))*[10]
        #torch.from_numpy numpy中的ndarray转为torch的tensor，反向操作.numpy()
        return img, torch.from_numpy(np.array(lbl[:5]))#只取5位，多的不要
    
    def __len__(self):
        return len(self.img_path)
            
            
            

In [228]:
#配置DataLoder 【设置 迭代器 】批量读取迭代方法['']
# glob.glob返回所有匹配的文件路径列表
train_path = glob.glob('data/mchar_train/*.png')
#print(train_path)#['data/mchar_train\\000000.png', 'data/mchar_train\\000001.png',
train_path.sort()
train_json = json.load(open('data/mchar_train.json'))
#print(train_json['000000.png'])#{'height': [219, 219], 'label': [1, 9], 'left': [246, 323], 'top': [77, 81], 'width': [81, 96]}
train_label = [train_json[x]['label'] for x in train_json]
print(train_label[:2])#[[1, 9], [2, 3]]
print(len(train_path),len(train_label))#查看x,y长度是否匹配

[[1, 9], [2, 3]]
30000 30000


In [229]:
train_loader = torch.utils.data.DataLoader(
    SVHNDataset(
        train_path,train_label,
        transforms.Compose([        #变形组件
            transforms.Resize((64,128)),
            #transforms.RandomCrop((60,120)),#随机区域剪裁
            #transforms.ColorJitter(0.3,0.3,0.2),#颜色的对比度、饱和度、亮度变换
            transforms.RandomRotation(10),#随机旋转
            transforms.ToTensor(),#ndarray变tensor
            transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])#归一化
        ])),
    batch_size = 40,#一组（批）多少个图（样本）
    shuffle = True,#打乱
    #num_workers = 10,#读取的线程个数
    num_workers = 0#改为单进程加载，后面报错Broken pipe
)
    
# for data in train_loader:
#     break

In [230]:
#调用DataLoder后，数据按照批次获取，每批次调用Dataset读取单个样本进行拼接
#data格式为x [10,3,64,128] 即[batchsize*chanel*height*width]y [10,6] [batchsize*标签结果值]

In [231]:
#验证集，同上处理
val_path = glob.glob('data/mchar_val/*.png')
val_path.sort()
val_json = json.load(open('data/mchar_val.json'))
val_label = [val_json[x]['label'] for x in val_json]
print(len(val_path),len(val_label))#查看x,y长度是否匹配

10000 10000


In [232]:
val_loader = torch.utils.data.DataLoader(
    SVHNDataset(
        val_path,val_label,
        transforms.Compose([        #变形组件
            transforms.Resize((60,128)),#120
            #transforms.RandomCrop((60,120)),#随机区域剪裁
            #transforms.ColorJitter(0.3,0.3,0.2),#颜色的对比度、饱和度、亮度变换
            #transforms.RandomRotation(10),#随机旋转
            transforms.ToTensor(),#ndarray变tensor
            transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])#归一化
        ])),
    batch_size = 40,#一组（批）多少个图（样本）
    shuffle = False,#打乱
    #num_workers = 10,#读取的线程个数
    num_workers = 0#改为单进程加载，后面报错Broken pipe
)

In [233]:
print(len(train_loader))
print(len(val_loader))
print(train_loader)
print(val_loader)

750
250


# task3-构建模型

In [234]:
#建模一：pytorch CNN 定义字符分类模型，用resnet预训练模型
class SVHN_Model_rsn(torch.nn.Module):
    def __init__(self):
        super(SVHN_Model_rsn,self).__init__()#继承torch.nn.Module类
        #resnet预训练实例化
        model_conv = models.resnet18(pretrained = True)
        model_conv.avgpool = nn.AdaptiveAvgPool2d(1)#自适应平均池化
        #取resnet最后一维结果集接自己的网络，继续训练，或接线性模型
        model_conv = torch.nn.Sequential(*list(model_conv.children())[:-1])
        self.cnn = model_conv
        
        #定义全连接层输出维度？，并行5次
        self.fc1 = torch.nn.Linear(512,11)
        self.fc2 = torch.nn.Linear(512,11)
        self.fc3 = torch.nn.Linear(512,11)
        self.fc4 = torch.nn.Linear(512,11)
        self.fc5 = torch.nn.Linear(512,11)
        
    #前向传播（神经网络一层一层往前走）    
    def forward(self,img):
        cnn_rs = self.cnn(img)#输入送入模型
        #view相当于tf的reshape，将前层的输出展平，去过全连接层，映射分类
        cnn_rs = cnn_rs.view(cnn_rs.shape[0],-1)
        #过上面定义的全连接层映射到定义的最终输出维度
        c1 = self.fc1(cnn_rs)
        c2 = self.fc2(cnn_rs)
        c3 = self.fc3(cnn_rs)
        c4 = self.fc4(cnn_rs)
        c5 = self.fc5(cnn_rs)
        
        return c1,c2,c3,c4,c5#返回分类结果
        

In [235]:
#建模二（备选）：pytorch CNN 定义字符分类模型，普通版，不用resnet预训练模型
class SVHN_Model_cnn(torch.nn.Module):
    def __init__(self):
        super(SVHN_Model_cnn,self).__init__()#继承torch.nn.Module类
        self.cnn = torch.nn.Sequential(
            
            torch.nn.Conv2d(3,16,kernel_size=(3,3),stride=(2, 2)),#卷积层 #3X64X128--> 16X31X63
            torch.nn.ReLU(),#过一个Relu激活函数不要写错，避免梯度爆炸
            torch.nn.MaxPool2d(2),#最大池化层，#16X31X63--> 16X15X31
            torch.nn.Conv2d(16,32,kernel_size = (3,3),stride = (2,2)),#再过一层卷积，16X15X31--> 32X7X15
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2)#32X7X15--> 32X3X7
        )
        
        #定义全连接层的输出维度，并行五次
        self.fc1 = torch.nn.Linear(32*3*7,11)
        self.fc2 = torch.nn.Linear(32*3*7,11)
        self.fc3 = torch.nn.Linear(32*3*7,11)
        self.fc4 = torch.nn.Linear(32*3*7,11)
        self.fc5 = torch.nn.Linear(32*3*7,11)
     
    #向前传播
    def forward(self,x):
        cnn_rs = self.cnn(x)
        cnn_rs = cnn_rs.view(cnn_rs.shape[0],-1)
        #过全连接层，映射输出
        f1 = self.fc1(cnn_rs)
        f2 = self.fc2(cnn_rs)
        f3 = self.fc3(cnn_rs)
        f4 = self.fc4(cnn_rs)
        f5 = self.fc5(cnn_rs)
        
        return f1,f2,f3,f4,f5
    

# task4 模型训练验证

In [236]:
#定义训练函数模块
def train(train_loader,model,criterion,optimizer):
    #切换到训练模式
    model.train()
    train_loss = []
    
    for i,(input, target) in enumerate(train_loader):#遍历数据
        if use_cuda:#gpu
            input = input.cuda()
            target = target.cuda()
        
        c1,c2,c3,c4,c5 = model(input)#模型输入得到输出预测值
        #评价，算Loss
        #预测值与真实值比较，计算loss
        loss = criterion(c1,target[:,0].long()) + criterion(c2,target[:,1].long()) + criterion(c3,target[:,2].long()) + criterion(c4,target[:,3].long()) + criterion(c5,target[:,4].long())
#报错：Expected object of scalar type Long but got scalar type Int for argument #2 'target'
#结果转long直接加.long即可
        # loss/=6 ?
        optimizer.zero_grad()#把所有Variable的grad成员数值变为0，优化器变量梯度清0，只算本步的加入loss列表
        loss.backward()#梯度反向传播
        optimizer.step()#更新梯度
        
        if i % 100 == 0:
            print(loss.item())#每隔100步打印loss
            
        train_loss.append(loss.item())#本步loss加入loss列表
        
    return np.mean(train_loss)#返回平均Loss
        

In [237]:
#定义验证模块
def validate(val_loader,model,criterion):
    #切换为预测模型
    model.eval()
    val_loss = []
    
    #不记录模型梯度信息，只记录Loss，用于打印
    with torch.no_grad():#加本句，数据不需要计算梯度，也不会进行反向传播
        for i, (input,target) in enumerate(val_loader):
            if use_cuda:
                input = input.cuda()
                target = target.cuda()
                
            c1,c2,c3,c4,c5 = model(input)
            loss = criterion(c1,target[:,0].long()) + criterion(c2,target[:,1].long()) + criterion(c3,target[:,2].long()) + criterion(c4,target[:,3].long())+ criterion(c5,target[:,4].long())
            val_loss.append(loss.item())
            
    return np.mean(val_loss)

In [238]:
#定义预测模块
def  predict(test_loader,model,tta=10):#tta次数
    #切换到预测模式
    model.eval()
    test_pred_tta = None
    
    #TTA次数
    for _ in range(tta):
        test_pred = []#预测结果列表
        
        with torch.no_grad():
            for i, (input, target) in enumerate(test_loader):
                if use_cuda:
                    input = input.cuda()
                    
                c1,c2,c3,c4,c5 = model(input)
                #预测结果拼接
                output = np.concatenate([
                    c1.data.numpy(),
                    c2.data.numpy(),
                    c3.data.numpy(),
                    c4.data.numpy(),
                    c5.data.numpy()
                ],axis = 1)#在列上拼接
                
                #加入结果集
                test_pred.append(output)
                
            #垂直方向堆叠数组
            if test_pred_tta is None:
                test_pred_tta = test_pred
            else:
                test_pred_tta += test_pred #叠加结果集
                
    return test_pred_tta      
        

In [ ]:
#训练
#model = SVHN_Model_rsn()#使用resnet模型
model = SVHN_Model_cnn()#使用无resnet的CNN模型
criterion = torch.nn.CrossEntropyLoss()#实例化评价函数
optimizer = torch.optim.Adam(model.parameters(),0.001)#实例化优化器
best_loss = 1000.0

use_cuda = False
if use_cuda:
    model = model.cuda()

for epoch in range(2):#训2轮
    #运行上面定义的训练函数模块
    train_loss = train(train_loader,model,criterion,optimizer)#epoch
    #运行上面定义的验证函数模块
    val_loss = validate(val_loader,model,criterion)
    
   
    
    #拼接真实结果集-图片标签，map(str,x)把字符串按字符拆开，join再拼接？格式处理
    val_label = [''.join(map(str,x)) for x in val_loader.dataset.img_label]
    val_predict_label = predict(val_loader,model,1)#预测标签结果
    #print("预测结果：",val_predict_label)
    #[array([[-2.0832968 ,  0.34928718,  0.63651973, ..., -4.029924  ,
     #   -4.017502  ,  3.8661182 ],

    '''
    #【【以下报错，slice不能是元组，待调】】
    #五组并行的预测结果集垂直方向拼接起来
    val_predict_label = np.vstack([
        val_predict_label[:,:11].argmax(1),#取预测概率最大索引的
        val_predict_label[:,11:22].argmax(1),
        val_predict_label[:,22:33].argmax(1),
        val_predict_label[:,33:44].argmax(1),
        val_predict_label[:,44:55].argmax(1),
        
    ]).T#转置
    
    #过滤掉类别10 pad补位符
    val_label_pred = []
    for x in val_predict_label:
        val_label_pred.append(''.join(map(str,x[x!=10])))
    
    #准确率，预测值==真实值的为准确
    val_char_acc = np.mean(np.array(val_label_pred)==np.array(val_label))
    
    print('Epoch: {0}, Train loss: {1} \t Val loss: {2}'.format(epoch,train_loss,val_loss))
    print(val_char_acc)
    
    '''
    
    #保存模型
    if val_loss < best_loss:#loss不再下降时保存模型
        best_loss = val_loss
        torch.save(model.state_dict(),'modelRs/model.pt')
    


In [ ]:
# 保存模型为'model.pt'
#torch.save(model_object.state_dict(), 'model.pt')
# 加载'model.pt'模型
#model.load_state_dict(torch.load(' model.pt'))

# 预测测试集，生成提交结果

In [244]:

#验证集，同上处理
test_path = glob.glob('data/mchar_test_a/*.png')
test_path.sort()
test_label = [[1]]*len(test_path)
print(len(val_path),len(val_label))#查看x,y长度是否匹配

10000 10000


In [245]:
test_loader = torch.utils.data.DataLoader(
    SVHNDataset(test_path, test_label,
                transforms.Compose([
                    transforms.Resize((64, 128)),
                    transforms.RandomCrop((60, 120)),
                    # transforms.ColorJitter(0.3, 0.3, 0.2),
                    # transforms.RandomRotation(5),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])), 
    batch_size=40, 
    shuffle=False, 
    num_workers=10,
)


In [246]:
print(len(test_loader))

1000


In [ ]:
#加载模型
model.load_state_dict(torch.load('modelRs/model.pt'))
test_predict_label = predict(test_loader,model,1)
print(test_predict_label.shape)
test_label = [''.join(map(str, x)) for x in test_loader.dataset.img_label]
test_predict_label = np.vstack([
    test_predict_label[:, :11].argmax(1),
    test_predict_label[:, 11:22].argmax(1),
    test_predict_label[:, 22:33].argmax(1),
    test_predict_label[:, 33:44].argmax(1),
    test_predict_label[:, 44:55].argmax(1),
]).T

test_label_pred = []
for x in test_predict_label:
    test_label_pred.append(''.join(map(str, x[x!=10])))
    
import pandas as pd
df_submit = pd.read_csv('rsData/test_A_sample_submit.csv')
df_submit['file_code'] = test_label_pred
df_submit.to_csv('pytorchCnn.csv', index=None)
